In [1]:
import carla
import random
import cv2
import numpy as np
import open3d as o3d
import matplotlib as plt
from matplotlib import cm
import math
import time
from ultralytics import YOLO
import torch
import tensorflow as tf
import tensorflow_hub as hub
import seaborn as sns

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#Connect to client
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

In [3]:
#load world
world = client.load_world('Town05')

In [4]:
# Initialize YOLO model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in C:\Users\Oz/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-11-17 Python-3.8.10 torch-2.4.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [5]:
#Spawn vehicles
def spawn_vehicles(world, num_vehicles):
    vehicles = []
    blueprints = world.get_blueprint_library().filter('vehicle.*')
    spawn_points = world.get_map().get_spawn_points()
    for _ in range(num_vehicles):
        spawn_point = random.choice(spawn_points)
        vehicle_bp = random.choice(blueprints)
        vehicle = world.try_spawn_actor(vehicle_bp, spawn_point)
        if vehicle is not None:
            vehicle.set_autopilot(True)
            vehicles.append(vehicle)
    return vehicles


#Spawn pedestrians
def spawn_pedestrians(world, client, num_pedestrians):
    pedestrians = []
    blueprint_library = world.get_blueprint_library()
    pedestrian_bps = blueprint_library.filter('walker.pedestrian.*')
    spawn_points = []
    for i in range(num_pedestrians):
        spawn_point = carla.Transform()
        loc = world.get_random_location_from_navigation()
        if loc is not None:
            spawn_point.location = loc
            spawn_points.append(spawn_point)
    batch = []
    for spawn_point in spawn_points:
        walker_bp = random.choice(pedestrian_bps)
        batch.append(carla.command.SpawnActor(walker_bp, spawn_point))
    pedestrians = client.apply_batch_sync(batch, True)
    pedestrians = [x.actor_id for x in pedestrians if not x.error]
    return pedestrians


#Lidar callback
def lidar_callback(point_cloud, point_list):
    global all_route_points
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
    data = np.reshape(data, (int(data.shape[0] / 4), 4))
    intensity = data[:, -1]
    intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))
    int_color = np.c_[
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 0]),
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 1]),
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 2])
    ]
    points = data[:, :-1]
    points[:, :1] = -points[:, :1]
    all_route_points.append(points)
    point_list.points = o3d.utility.Vector3dVector(points)
    point_list.colors = o3d.utility.Vector3dVector(int_color)


#Calculate velocity
def calculate_velocity(d_p):
    v_max = 15
    theta_2 = 0.3
    theta_3 = 3
    v = v_max * (1 - math.exp(-theta_2 * (d_p - theta_3)))
    return v


#Radar callback
def radar_callback(data):
    flag = 0
    closest_distance = float('inf')
    for detection in data:
        distance = detection.depth
        if distance < closest_distance:
            closest_distance = distance
    velocity = calculate_velocity(closest_distance)
    velocity_speed = max(0, velocity)
    if flag % 40 == 0:
        print(f"Closest object distance: {closest_distance:.1f} meters")
        print(f"Calculated velocity: {velocity_speed:.1f} m/s")
    flag += 1


#Add a small 3D axis on Open3D Visualizer
def add_open3d_axis(vis):
    axis = o3d.geometry.LineSet()
    axis.points = o3d.utility.Vector3dVector(np.array([
        [0.0, 0.0, 0.0],
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]
    ]))
    axis.lines = o3d.utility.Vector2iVector(np.array([
        [0, 1],
        [0, 2],
        [0, 3]
    ]))
    axis.colors = o3d.utility.Vector3dVector(np.array([
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]
    ]))
    vis.add_geometry(axis)


#Image process for traffic object classification
def image_process(frame):
    traffic_classes = [0, 1, 2, 3, 5, 7, 9, 10, 13, 14, 16]  # [person, bicycle, car, motorbike, bus, train, truck, traffic light, fire hydrant, stop sign, parking meter]
    results = model(frame)
    detections = results.xyxy[0].cpu().numpy()
    if len(detections) == 0:
        return frame
    for *box, conf, cls in detections:
        cls = int(cls)
        if cls in traffic_classes:
            x1, y1, x2, y2 = map(int, box)
            conf = round(float(conf), 2)
            color = (0, 255, 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            label = f'{model.names[cls]}'
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, label, (x1, y1 - 10), font, 0.5, color, 2)
    return frame


#Create main vehicle
def create_hero_vehicle():
    vehicle_bp = world.get_blueprint_library().filter('*mini*')
    spawn_points = world.get_map().get_spawn_points()
    spawn_point = random.choice(spawn_points)
    hero_vehicle = world.try_spawn_actor(vehicle_bp[0], spawn_point)
    hero_vehicle.set_autopilot(True)
    return hero_vehicle


#Camera callback
def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))


#Remove all vehicles
def remove_all_vehicles(world):
    actors = world.get_actors()
    for actor in actors:
        if actor.type_id.startswith('vehicle.'):
            actor.destroy()  


#Remove all pedestrians
def remove_all_pedestrians(world):
    actors = world.get_actors()
    for actor in actors:
        if actor.type_id.startswith('walker.pedestrian.'):
            actor.destroy()

In [6]:
#Call functions
pedestrians = spawn_pedestrians(world, client, 100)
vehicles = spawn_vehicles(world, 99)
hero_vehicle = create_hero_vehicle()

In [7]:
#Create and attach sensors to main vehicle
all_route_points = []
VIRIDIS = np.array(plt.colormaps['plasma'].colors)
VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
COOL_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
COOL = np.array(plt.colormaps['winter'](COOL_RANGE))
COOL = COOL[:, :3]

CAMERA_POS_Z = 1.6
CAMERA_POS_X = 0.9

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1152')
camera_bp.set_attribute('image_size_y', '648')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=hero_vehicle)

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}

lidar_bp = world.get_blueprint_library().find('sensor.lidar.ray_cast')
lidar_bp.set_attribute('range', '50.0')
lidar_bp.set_attribute('noise_stddev', '0.1')
lidar_bp.set_attribute('upper_fov', '10.0')
lidar_bp.set_attribute('lower_fov', '-3.0')
lidar_bp.set_attribute('channels', '32')
lidar_bp.set_attribute('rotation_frequency', '60.0')
lidar_bp.set_attribute('points_per_second', '200000')
lidar_init_trans = carla.Transform(carla.Location(z=2))
lidar = world.spawn_actor(lidar_bp, lidar_init_trans, attach_to=hero_vehicle)

radar_bp = world.get_blueprint_library().find('sensor.other.radar')
radar_bp.set_attribute('horizontal_fov', '30.0')
radar_bp.set_attribute('vertical_fov', '30.0')
radar_bp.set_attribute('points_per_second', '10000')
radar_init_trans = carla.Transform(carla.Location(z=2))
radar = world.spawn_actor(radar_bp, radar_init_trans, attach_to=hero_vehicle)

point_list = o3d.geometry.PointCloud()

camera.listen(lambda image: camera_callback(image,camera_data))
lidar.listen(lambda data: lidar_callback(data, point_list))

In [8]:
#Show lidar and camera outputs & print velocity control and object distance from radar outputs
vis = o3d.visualization.Visualizer()
vis.create_window(
    window_name='Carla Lidar',
    width=960,
    height=540,
    left=480,
    top=270
)
vis.get_render_option().background_color = [0.05, 0.05, 0.05]
vis.get_render_option().point_size = 1
vis.get_render_option().show_coordinate_frame = True
add_open3d_axis(vis)
spectator = world.get_spectator()
radar.listen(lambda data: radar_callback(data))
while True:
    # Accumulate points in combined point cloud
    transform = carla.Transform(hero_vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)), hero_vehicle.get_transform().rotation)
    spectator.set_transform(transform)
    vis.add_geometry(point_list)
    vis.update_geometry(point_list)
    vis.poll_events()
    vis.update_renderer()
    frame = camera_data['image']
    frame = image_process(frame)
    cv2.imshow('RGB Camera',frame)
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()
camera.stop()
camera.destroy()
radar.stop()
radar.destroy()
lidar.stop()
lidar.destroy()
vis.destroy_window()

Closest object distance: 8.0 meters
Calculated velocity: 11.6 m/s
Closest object distance: 8.2 meters
Calculated velocity: 11.8 m/s
Closest object distance: 8.2 meters
Calculated velocity: 11.9 m/s
Closest object distance: 8.2 meters
Calculated velocity: 11.9 m/s
Closest object distance: 8.3 meters
Calculated velocity: 11.9 m/s
Closest object distance: 8.5 meters
Calculated velocity: 12.1 m/s
Closest object distance: 7.9 meters
Calculated velocity: 11.5 m/s


C:\Users\Oz/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Oz/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Oz/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Closest object distance: 8.1 meters
Calculated velocity: 11.7 m/s
Closest object distance: 8.0 meters
Calculated velocity: 11.6 m/s
Closest object distance: 7.9 meters
Calculated velocity: 11.6 m/s
Closest object distance: 8.0 meters
Calculated velocity: 11.6 m/s
Closest object distance: 8.4 meters
Calculated velocity: 12.1 m/s
Closest object distance: 8.7 meters
Calculated velocity: 12.3 m/s
Closest object distance: 8.2 meters
Calculated velocity: 11.9 m/s
Closest object distance: 8.5 meters
Calculated velocity: 12.1 m/s
Closest object distance: 8.5 meters
Calculated velocity: 12.1 m/s
Closest object distance: 8.3 meters
Calculated velocity: 12.0 m/s
Closest object distance: 8.2 meters
Calculated velocity: 11.9 m/s
Closest object distance: 8.3 meters
Calculated velocity: 12.0 m/s
Closest object distance: 8.1 meters
Calculated velocity: 11.8 m/s


C:\Users\Oz/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Closest object distance: 8.1 meters
Calculated velocity: 11.7 m/s
Closest object distance: 8.4 meters
Calculated velocity: 12.0 m/s
Closest object distance: 8.1 meters
Calculated velocity: 11.8 m/s
Closest object distance: 8.2 meters
Calculated velocity: 11.8 m/s
Closest object distance: 8.4 meters
Calculated velocity: 12.1 m/s
Closest object distance: 8.5 meters
Calculated velocity: 12.1 m/s


...

Closest object distance: 8.2 meters
Calculated velocity: 11.8 m/s
Closest object distance: 8.1 meters
Calculated velocity: 11.7 m/s
Closest object distance: 7.9 meters
Calculated velocity: 11.6 m/s
Closest object distance: 8.1 meters
Calculated velocity: 11.7 m/s
Closest object distance: 8.4 meters
Calculated velocity: 12.0 m/s
Closest object distance: 8.5 meters
Calculated velocity: 12.1 m/s
Closest object distance: 8.1 meters
Calculated velocity: 11.7 m/s
Closest object distance: 8.6 meters
Calculated velocity: 12.2 m/s


C:\Users\Oz/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Closest object distance: 8.4 meters
Calculated velocity: 12.0 m/s
Closest object distance: 8.0 meters
Calculated velocity: 11.7 m/s
Closest object distance: 7.9 meters
Calculated velocity: 11.6 m/s
Closest object distance: 8.2 meters
Calculated velocity: 11.9 m/s
Closest object distance: 9.0 meters
Calculated velocity: 12.5 m/s
Closest object distance: 8.2 meters
Calculated velocity: 11.9 m/s
Closest object distance: 8.4 meters
Calculated velocity: 12.0 m/s
Closest object distance: 8.4 meters
Calculated velocity: 12.1 m/s
Closest object distance: 8.4 meters
Calculated velocity: 12.0 m/s
Closest object distance: 8.1 meters
Calculated velocity: 11.7 m/s
Closest object distance: 8.0 meters
Calculated velocity: 11.6 m/s
Closest object distance: 8.3 meters
Calculated velocity: 12.0 m/s
Closest object distance: 7.9 meters
Calculated velocity: 11.6 m/s


C:\Users\Oz/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Closest object distance: 8.3 meters
Calculated velocity: 12.0 m/s
Closest object distance: 8.4 meters
Calculated velocity: 12.0 m/s


In [9]:
# Combine all point clouds into a single large point cloud
combined_point_cloud = o3d.geometry.PointCloud()
for points in all_route_points:
    pc = o3d.geometry.PointCloud()
    pc.points = o3d.utility.Vector3dVector(points)
    combined_point_cloud += pc
    
print("Point Cloud before downsampling: ",len(combined_point_cloud.points))
downsampled_cloud  = combined_point_cloud.voxel_down_sample(voxel_size=0.2)
downsampled_cloud = downsampled_cloud.uniform_down_sample(30)
downsampled_cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
cl, ind = downsampled_cloud.remove_statistical_outlier(nb_neighbors=30, std_ratio=1.2)
cleaned_cloud = downsampled_cloud.select_by_index(ind)
print("Point Cloud after downsampling: ",len(cleaned_cloud.points))

Point Cloud before downsampling:  7516339
Point Cloud after downsampling:  36041


In [10]:
#Remove all actors
remove_all_vehicles(world)
remove_all_pedestrians(world)